In [1]:
try:
    # Check if running in Google Colab
    from google.colab import drive
    import os

    # Mount Google Drive
    drive.mount('/content/drive')

    # Change directory to Google Drive
    os.chdir('/content/drive/My Drive/Detecting_Fake_Accounts')
    !pip install deepface
    # Now you're in Google Colab environment, execute further code if needed
    # For example, cloning a repository or working with files in Google Drive
    # Remember to prepend '!' to execute shell commands in Colab
except ImportError:
    # If not in Colab, print a message or handle it in another way
    print("This code is meant to run in Google Colab.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
from unique_name_identifiers import split_string, get_names, get_name_list
from comp_vision_module import compare_faces
from NLI_Module import impersonation_detection, fan_tribute_detection

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


First the "unique name identifiers" need to be added to each account

In [3]:
df = pd.read_csv("Social_media_profiles - update.csv")

First extract a list of verified names

In [4]:
official_names = get_name_list(df)

In [5]:
official_names

['jeff',
 'elon',
 'jennifer',
 'lisa',
 'yann',
 'beyoncé',
 'ariana',
 'leonardo',
 'max',
 'andrew',
 'geoffrey',
 'demis',
 'fei-fei',
 'daphne',
 'bezos',
 'musk',
 'widom',
 'randall',
 'lecun',
 'grande',
 'dicaprio',
 'tegmark',
 'ng',
 'hinton',
 'hassabis',
 'li',
 'koller']

Next, We will obtain the unique name identifers differently for verified accounts and non-verified accounts. The idea is that for verified accounts we will require them to provide first and last names as associated with the account so as to easily get a unique name identifier. For non-verified we will take a more creative approach for filtering out uneccessary characters and symbols and just getting the first and last names.

In [6]:
df['Account_identifier'] = np.where(df['Verified']==True,df['First_Name'].str.lower() + df['Last_Name'].str.lower(),df['User_Name'].apply(split_string).apply(lambda x: get_names(x,official_names)))

In [7]:
df

,Person_id,User_Name,First_Name,Last_Name,Bio,Profile_photo_address,Verified,Ground_truth_label,Account_identifier
0,1,Jeff Bezos,Jeff,Bezos,Amazon. Blue Origin. Washington Post. Bezos Ea...,https://pbs.twimg.com/profile_images/159155831...,True,Real,jeffbezos
1,2,Jeff Bezos Private ,NaN,NaN,Amazon. Blue Origin. Washington Post. Bezos Ea...,https://pbs.twimg.com/profile_images/170164714...,False,Fake,jeffbezos
2,3,Elon Musk,Elon,Musk,NaN,https://pbs.twimg.com/profile_images/178004448...,True,Real,elonmusk
3,4,Jennifer Widom,Jennifer,Widom,Dean @StanfordEng,http://cs.stanford.edu/people/widom/photos/por...,True,Real,jenniferwidom
4,5,Lisa Randall,Lisa,Randall,Physicist and author. Dark Matter and the Dino...,https://pbs.twimg.com/profile_images/476919936...,True,Real,lisarandall
5,6,Yann LeCun,Yann,LeCun,Professor at NYU. Chief AI Scientist at Meta.\...,https://pbs.twimg.com/profile_images/148357786...,True,Real,yannlecun
6,7,Elon Musk (Memes),NaN,NaN,This account is not affiliated with the real E...,https://pbs.twimg.com/profile_images/174140309...,False,Fan,elonmusk
7,8,Dr. Yann LeCun,NaN,NaN,Head of AI at Meta,https://upload.wikimedia.org/wikipedia/commons...,False,Fake,yannlecun
8,9,BEYONCÉ,BEYONCÉ,NaN,NaN,https://pbs.twimg.com/profile_images/177011918...,True,Real,NaN
9,10,Ariana Grande Today,NaN,NaN,the #1 fan source for the latest ariana grande...,https://pbs.twimg.com/profile_images/176614057...,False,Fan,arianagrande


In [8]:
comparison_df = pd.merge(df[df["Verified"]==True],df[df["Verified"]==False],on = 'Account_identifier' ,how='inner')

In [9]:
comparison_df

,Person_id_x,User_Name_x,First_Name_x,Last_Name_x,Bio_x,Profile_photo_address_x,Verified_x,Ground_truth_label_x,Account_identifier,Person_id_y,User_Name_y,First_Name_y,Last_Name_y,Bio_y,Profile_photo_address_y,Verified_y,Ground_truth_label_y
0,1,Jeff Bezos,Jeff,Bezos,Amazon. Blue Origin. Washington Post. Bezos Ea...,https://pbs.twimg.com/profile_images/159155831...,True,Real,jeffbezos,2,Jeff Bezos Private ,NaN,NaN,Amazon. Blue Origin. Washington Post. Bezos Ea...,https://pbs.twimg.com/profile_images/170164714...,False,Fake
1,3,Elon Musk,Elon,Musk,NaN,https://pbs.twimg.com/profile_images/178004448...,True,Real,elonmusk,7,Elon Musk (Memes),NaN,NaN,This account is not affiliated with the real E...,https://pbs.twimg.com/profile_images/174140309...,False,Fan
2,4,Jennifer Widom,Jennifer,Widom,Dean @StanfordEng,http://cs.stanford.edu/people/widom/photos/por...,True,Real,jenniferwidom,26,Real Jennifer Widom,NaN,NaN,Studying microbiology at Stanford,https://hackspirit.com/wp-content/uploads/2021...,False,Real
3,6,Yann LeCun,Yann,LeCun,Professor at NYU. Chief AI Scientist at Meta.\...,https://pbs.twimg.com/profile_images/148357786...,True,Real,yannlecun,8,Dr. Yann LeCun,NaN,NaN,Head of AI at Meta,https://upload.wikimedia.org/wikipedia/commons...,False,Fake
4,11,Ariana Grande,Ariana,Grande,eternal sunshine ˚❀*·ꕤ. out now,https://upload.wikimedia.org/wikipedia/commons...,True,Real,arianagrande,10,Ariana Grande Today,NaN,NaN,the #1 fan source for the latest ariana grande...,https://pbs.twimg.com/profile_images/176614057...,False,Fan
5,12,Leonardo DiCaprio,Leonardo,DiCaprio,Actor and Environmentalist,https://pbs.twimg.com/profile_images/694662257...,True,Real,leonardodicaprio,18,Leonardo DiCaprio,NaN,NaN,Actor and Environmentalist,https://pbs.twimg.com/profile_images/174692206...,False,Fake
6,13,Max Tegmark,Max,Tegmark,Known as Mad Max for my unorthodox ideas and p...,https://pbs.twimg.com/profile_images/471772020...,True,Real,maxtegmark,15,Max Tegmark,NaN,NaN,professor at the Massachusetts Institute of Te...,https://upload.wikimedia.org/wikipedia/commons...,False,Fake
7,16,Andrew Ng,Andrew,Ng,Co-Founder of Coursera; Stanford CS adjunct fa...,https://pbs.twimg.com/profile_images/733174243...,True,Real,andrewng,14,Andrew Ng,NaN,NaN,Co-founder of coursera: Stanford CS adjunct fa...,https://pbs.twimg.com/profile_images/167856189...,False,Fake
8,17,Geoffrey Hinton,Geoffrey,Hinton,deep learning,https://pbs.twimg.com/profile_images/108421331...,True,Real,geoffreyhinton,25,Geoffrey Hinton,NaN,NaN,"Investment Banker, Dad, Eagles fan!",https://neweralive.na/storage/images/2023/may/...,False,Real
9,19,Demis Hassabis,Demis,Hassabis,Co-founder & CEO @GoogleDeepMind - working on ...,https://pbs.twimg.com/profile_images/691700243...,True,Real,demishassabis,20,Demis Hassabis Official,NaN,NaN,Co-founder & CEO @GoogleDeepMind,https://assets.bizclikmedia.net/1336/c8201edf0...,False,Fake


next columns will be created containing data from the verified and potentially fake profiles for comparison

In [10]:
comparison_df['profile_pic_comparison'] = list(zip(comparison_df['Profile_photo_address_x'], comparison_df['Profile_photo_address_y']))

In [11]:
comparison_df['tribute_comparison'] = list(zip(comparison_df['User_Name_x'], comparison_df['Bio_y']))

In [12]:
comparison_df['fraud_comparison'] = list(zip(comparison_df['Bio_x'], comparison_df['Bio_y']))

In [13]:
comparison_df

,Person_id_x,User_Name_x,First_Name_x,Last_Name_x,Bio_x,Profile_photo_address_x,Verified_x,Ground_truth_label_x,Account_identifier,Person_id_y,User_Name_y,First_Name_y,Last_Name_y,Bio_y,Profile_photo_address_y,Verified_y,Ground_truth_label_y,profile_pic_comparison,tribute_comparison,fraud_comparison
0,1,Jeff Bezos,Jeff,Bezos,Amazon. Blue Origin. Washington Post. Bezos Ea...,https://pbs.twimg.com/profile_images/159155831...,True,Real,jeffbezos,2,Jeff Bezos Private ,NaN,NaN,Amazon. Blue Origin. Washington Post. Bezos Ea...,https://pbs.twimg.com/profile_images/170164714...,False,Fake,(https://pbs.twimg.com/profile_images/15915583...,"(Jeff Bezos, Amazon. Blue Origin. Washington P...",(Amazon. Blue Origin. Washington Post. Bezos E...
1,3,Elon Musk,Elon,Musk,NaN,https://pbs.twimg.com/profile_images/178004448...,True,Real,elonmusk,7,Elon Musk (Memes),NaN,NaN,This account is not affiliated with the real E...,https://pbs.twimg.com/profile_images/174140309...,False,Fan,(https://pbs.twimg.com/profile_images/17800444...,"(Elon Musk, This account is not affiliated wit...","(nan, This account is not affiliated with the ..."
2,4,Jennifer Widom,Jennifer,Widom,Dean @StanfordEng,http://cs.stanford.edu/people/widom/photos/por...,True,Real,jenniferwidom,26,Real Jennifer Widom,NaN,NaN,Studying microbiology at Stanford,https://hackspirit.com/wp-content/uploads/2021...,False,Real,(http://cs.stanford.edu/people/widom/photos/po...,"(Jennifer Widom, Studying microbiology at Stan...","(Dean @StanfordEng, Studying microbiology at S..."
3,6,Yann LeCun,Yann,LeCun,Professor at NYU. Chief AI Scientist at Meta.\...,https://pbs.twimg.com/profile_images/148357786...,True,Real,yannlecun,8,Dr. Yann LeCun,NaN,NaN,Head of AI at Meta,https://upload.wikimedia.org/wikipedia/commons...,False,Fake,(https://pbs.twimg.com/profile_images/14835778...,"(Yann LeCun, Head of AI at Meta)",(Professor at NYU. Chief AI Scientist at Meta....
4,11,Ariana Grande,Ariana,Grande,eternal sunshine ˚❀*·ꕤ. out now,https://upload.wikimedia.org/wikipedia/commons...,True,Real,arianagrande,10,Ariana Grande Today,NaN,NaN,the #1 fan source for the latest ariana grande...,https://pbs.twimg.com/profile_images/176614057...,False,Fan,(https://upload.wikimedia.org/wikipedia/common...,"(Ariana Grande, the #1 fan source for the late...","(eternal sunshine ˚❀*·ꕤ. out now, the #1 fan s..."
5,12,Leonardo DiCaprio,Leonardo,DiCaprio,Actor and Environmentalist,https://pbs.twimg.com/profile_images/694662257...,True,Real,leonardodicaprio,18,Leonardo DiCaprio,NaN,NaN,Actor and Environmentalist,https://pbs.twimg.com/profile_images/174692206...,False,Fake,(https://pbs.twimg.com/profile_images/69466225...,"(Leonardo DiCaprio, Actor and Environmentalist)","(Actor and Environmentalist, Actor and Environ..."
6,13,Max Tegmark,Max,Tegmark,Known as Mad Max for my unorthodox ideas and p...,https://pbs.twimg.com/profile_images/471772020...,True,Real,maxtegmark,15,Max Tegmark,NaN,NaN,professor at the Massachusetts Institute of Te...,https://upload.wikimedia.org/wikipedia/commons...,False,Fake,(https://pbs.twimg.com/profile_images/47177202...,"(Max Tegmark, professor at the Massachusetts I...",(Known as Mad Max for my unorthodox ideas and ...
7,16,Andrew Ng,Andrew,Ng,Co-Founder of Coursera; Stanford CS adjunct fa...,https://pbs.twimg.com/profile_images/733174243...,True,Real,andrewng,14,Andrew Ng,NaN,NaN,Co-founder of coursera: Stanford CS adjunct fa...,https://pbs.twimg.com/profile_images/167856189...,False,Fake,(https://pbs.twimg.com/profile_images/73317424...,"(Andrew Ng, Co-founder of coursera: Stanford C...",(Co-Founder of Coursera; Stanford CS adjunct f...
8,17,Geoffrey Hinton,Geoffrey,Hinton,deep learning,https://pbs.twimg.com/profile_images/108421331...,True,Real,geoffreyhinton,25,Geoffrey Hinton,NaN,NaN,"Investment Banker, Dad, Eagles fan!",https://neweralive.na/storage/images/2023/may/...,False,Real,(https://pbs.twimg.com/profile_images/10842133...,"(Geoffrey Hinton, Investment Banker, Dad, Eagl...","(deep learni

In [14]:
comparison_df['fan tribute liklihood'] = comparison_df['tribute_comparison'].apply(lambda x: fan_tribute_detection(x[0],x[1]))

In [15]:
comparison_df['impersonation liklihood'] = comparison_df['fraud_comparison'].apply(lambda x: impersonation_detection(x[0],x[1]))

In [16]:
comparison_df

,Person_id_x,User_Name_x,First_Name_x,Last_Name_x,Bio_x,Profile_photo_address_x,Verified_x,Ground_truth_label_x,Account_identifier,Person_id_y,...,Last_Name_y,Bio_y,Profile_photo_address_y,Verified_y,Ground_truth_label_y,profile_pic_comparison,tribute_comparison,fraud_comparison,fan tribute liklihood,impersonation liklihood
0,1,Jeff Bezos,Jeff,Bezos,Amazon. Blue Origin. Washington Post. Bezos Ea...,https://pbs.twimg.com/profile_images/159155831...,True,Real,jeffbezos,2,...,NaN,Amazon. Blue Origin. Washington Post. Bezos Ea...,https://pbs.twimg.com/profile_images/170164714...,False,Fake,(https://pbs.twimg.com/profile_images/15915583...,"(Jeff Bezos, Amazon. Blue Origin. Washington P...",(Amazon. Blue Origin. Washington Post. Bezos E...,0.751429,0.754274
1,3,Elon Musk,Elon,Musk,NaN,https://pbs.twimg.com/profile_images/178004448...,True,Real,elonmusk,7,...,NaN,This account is not affiliated with the real E...,https://pbs.twimg.com/profile_images/174140309...,False,Fan,(https://pbs.twimg.com/profile_images/17800444...,"(Elon Musk, This account is not affiliated wit...","(nan, This account is not affiliated with the ...",0.988440,NaN
2,4,Jennifer Widom,Jennifer,Widom,Dean @StanfordEng,http://cs.stanford.edu/people/widom/photos/por...,True,Real,jenniferwidom,26,...,NaN,Studying microbiology at Stanford,https://hackspirit.com/wp-content/uploads/2021...,False,Real,(http://cs.stanford.edu/people/widom/photos/po...,"(Jennifer Widom, Studying microbiology at Stan...","(Dean @StanfordEng, Studying microbiology at S...",0.034540,0.330143
3,6,Yann LeCun,Yann,LeCun,Professor at NYU. Chief AI Scientist at Meta.\...,https://pbs.twimg.com/profile_images/148357786...,True,Real,yannlecun,8,...,NaN,Head of AI at Meta,https://upload.wikimedia.org/wikipedia/commons...,False,Fake,(https://pbs.twimg.com/profile_images/14835778...,"(Yann LeCun, Head of AI at Meta)",(Professor at NYU. Chief AI Scientist at Meta....,0.068275,0.789896
4,11,Ariana Grande,Ariana,Grande,eternal sunshine ˚❀*·ꕤ. out now,https://upload.wikimedia.org/wikipedia/commons...,True,Real,arianagrande,10,...,NaN,the #1 fan source for the latest ariana grande...,https://pbs.twimg.com/profile_images/176614057...,False,Fan,(https://upload.wikimedia.org/wikipedia/common...,"(Ariana Grande, the #1 fan source for the late...","(eternal sunshine ˚❀*·ꕤ. out now, the #1 fan s...",0.314406,0.742407
5,12,Leonardo DiCaprio,Leonardo,DiCaprio,Actor and Environmentalist,https://pbs.twimg.com/profile_images/694662257...,True,Real,leonardodicaprio,18,...,NaN,Actor and Environmentalist,https://pbs.twimg.com/profile_images/174692206...,False,Fake,(https://pbs.twimg.com/profile_images/69466225...,"(Leonardo DiCaprio, Actor and Environmentalist)","(Actor and Environmentalist, Actor and Environ...",0.730446,0.988528
6,13,Max Tegmark,Max,Tegmark,Known as Mad Max for my unorthodox ideas and p...,https://pbs.twimg.com/profile_images/471772020...,True,Real,maxtegmark,15,...,NaN,professor at the Massachusetts Institute of Te...,https://upload.wikimedia.org/wikipedia/commons...,False,Fake,(https://pbs.twimg.com/profile_images/47177202...,"(Max Tegmark, professor at the Massachusetts I...",(Known as Mad Max for my unorthodox ideas and ...,0.587975,0.518524
7,16,Andrew Ng,Andrew,Ng,Co-Founder of Coursera; Stanford CS adjunct fa...,https://pbs.twimg.com/profile_images/733174243...,True,Real,andrewng,14,...,NaN,Co-founder of coursera: Stanford CS adjunct fa...,https://pbs.twimg.com/profile_images/167856189...,False,Fake,(https://pbs.twimg.com/profile_images/73317424...,"(Andrew Ng, Co-founder of coursera: Stanford C...",(Co-Founder of Coursera; Stanford CS adjunct f...,0.841883,0.618231
8,17,Geoffrey Hinton,Geoffrey,Hinton,deep learning,https://pbs.twimg.com/profile_images/108421331...,True,Real,geoffreyhinton,25,...,NaN,"Investment Banker, Dad, Eagles fan!",https://neweralive.na/storage/images/2023/may/...,False,Real,(https://pbs.twimg.com/profile_images/10842133...,"(Geoffrey Hinton, Investment Banker, Dad, Eagl..."

In [16]:
comparison_df['comparison_score'] = comparison_df['profile_pic_comparison'].apply(lambda x: compare_faces(x[0],x[1]))

24-05-07 20:03:30 - vgg_face_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5
To: /root/.deepface/weights/vgg_face_weights.h5
100%|██████████| 580M/580M [00:01<00:00, 451MB/s]


24-05-07 20:03:37 - deploy.prototxt will be downloaded...


Downloading...
From: https://github.com/opencv/opencv/raw/3.4.0/samples/dnn/face_detector/deploy.prototxt
To: /root/.deepface/weights/deploy.prototxt
28.1kB [00:00, 17.8MB/s]                   


24-05-07 20:03:38 - res10_300x300_ssd_iter_140000.caffemodel will be downloaded...


...

24-05-07 20:04:17 - face_detection_yunet_2023mar.onnx will be downloaded...


Downloading...
From: https://github.com/opencv/opencv_zoo/raw/main/models/face_detection_yunet/face_detection_yunet_2023mar.onnx
To: /root/.deepface/weights/face_detection_yunet_2023mar.onnx
100%|██████████| 233k/233k [00:00<00:00, 8.82MB/s]


24-05-07 20:04:18 - Downloading CenterFace weights from https://github.com/Star-Clouds/CenterFace/raw/master/models/onnx/centerface.onnx to /root/.deepface/weights/centerface.onnx...


Downloading...
From: https://github.com/Star-Clouds/CenterFace/raw/master/models/onnx/centerface.onnx
To: /root/.deepface/weights/centerface.onnx
100%|██████████| 7.53M/7.53M [00:00<00:00, 131MB/s]


24-05-07 20:04:20 - CenterFace model is just downloaded to centerface.onnx
1/1 [==============================] - 0s 18ms/step


In [17]:
comparison_df

,Person_id_x,User_Name_x,First_Name_x,Last_Name_x,Bio_x,Profile_photo_address_x,Verified_x,Ground_truth_label_x,Account_identifier,Person_id_y,User_Name_y,First_Name_y,Last_Name_y,Bio_y,Profile_photo_address_y,Verified_y,Ground_truth_label_y,profile_pic_comparison,comparison_score
0,1,Jeff Bezos,Jeff,Bezos,Amazon. Blue Origin. Washington Post. Bezos Ea...,https://pbs.twimg.com/profile_images/159155831...,True,Real,jeffbezos,2,Jeff Bezos Private ,NaN,NaN,Amazon. Blue Origin. Washington Post. Bezos Ea...,https://pbs.twimg.com/profile_images/170164714...,False,Fake,(https://pbs.twimg.com/profile_images/15915583...,0.001331
1,3,Elon Musk,Elon,Musk,NaN,https://pbs.twimg.com/profile_images/178004448...,True,Real,elonmusk,7,Elon Musk (Memes),NaN,NaN,This account is not affiliated with the real E...,https://pbs.twimg.com/profile_images/174140309...,False,Fan,(https://pbs.twimg.com/profile_images/17800444...,0.496664
2,4,Jennifer Widom,Jennifer,Widom,Dean @StanfordEng,http://cs.stanford.edu/people/widom/photos/por...,True,Real,jenniferwidom,26,Real Jennifer Widom,NaN,NaN,Studying microbiology at Stanford,https://hackspirit.com/wp-content/uploads/2021...,False,Real,(http://cs.stanford.edu/people/widom/photos/po...,NaN
3,6,Yann LeCun,Yann,LeCun,Professor at NYU. Chief AI Scientist at Meta.\...,https://pbs.twimg.com/profile_images/148357786...,True,Real,yannlecun,8,Dr. Yann LeCun,NaN,NaN,Head of AI at Meta,https://upload.wikimedia.org/wikipedia/commons...,False,Fake,(https://pbs.twimg.com/profile_images/14835778...,0.673636
4,11,Ariana Grande,Ariana,Grande,eternal sunshine ˚❀*·ꕤ. out now,https://upload.wikimedia.org/wikipedia/commons...,True,Real,arianagrande,10,Ariana Grande Today,NaN,NaN,the #1 fan source for the latest ariana grande...,https://pbs.twimg.com/profile_images/176614057...,False,Fan,(https://upload.wikimedia.org/wikipedia/common...,0.568673
5,12,Leonardo DiCaprio,Leonardo,DiCaprio,Actor and Environmentalist,https://pbs.twimg.com/profile_images/694662257...,True,Real,leonardodicaprio,18,Leonardo DiCaprio,NaN,NaN,Actor and Environmentalist,https://pbs.twimg.com/profile_images/174692206...,False,Fake,(https://pbs.twimg.com/profile_images/69466225...,0.061660
6,13,Max Tegmark,Max,Tegmark,Known as Mad Max for my unorthodox ideas and p...,https://pbs.twimg.com/profile_images/471772020...,True,Real,maxtegmark,15,Max Tegmark,NaN,NaN,professor at the Massachusetts Institute of Te...,https://upload.wikimedia.org/wikipedia/commons...,False,Fake,(https://pbs.twimg.com/profile_images/47177202...,0.496895
7,16,Andrew Ng,Andrew,Ng,Co-Founder of Coursera; Stanford CS adjunct fa...,https://pbs.twimg.com/profile_images/733174243...,True,Real,andrewng,14,Andrew Ng,NaN,NaN,Co-founder of coursera: Stanford CS adjunct fa...,https://pbs.twimg.com/profile_images/167856189...,False,Fake,(https://pbs.twimg.com/profile_images/73317424...,0.063184
8,17,Geoffrey Hinton,Geoffrey,Hinton,deep learning,https://pbs.twimg.com/profile_images/108421331...,True,Real,geoffreyhinton,25,Geoffrey Hinton,NaN,NaN,"Investment Banker, Dad, Eagles fan!",https://neweralive.na/storage/images/2023/may/...,False,Real,(https://pbs.twimg.com/profile_images/10842133...,0.880035
9,19,Demis Hassabis,Demis,Hassabis,Co-founder & CEO @GoogleDeepMind - working on ...,https://pbs.twimg.com/profile_images/691700243...,True,Real,demishassabis,20,Demis Hassabis Official,NaN,NaN,Co-founder & CEO @GoogleDeepMind,https://assets.bizclikmedia.net/1336/c8201edf0...,False,Fake,(https://pbs.twimg.com/profile_images/69170024...,0.339955
